# IBM Data Science - Capstone Project

### ~ importing required packages

In [1]:
import pandas as pd
import numpy as np

import random

import requests
import bs4 as BeautifulSoup
import json
from pandas.io.json import json_normalize

import plotly.plotly as py
import plotly.graph_objs as go

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn import metrics
#from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

print("Packages are imported!")

Packages are imported!


### ~ a function to collect data from url

In [2]:
def htmltodf(url, table_num = 0, add_col = 0):
    
    html_doc = requests.get(url)

    file = open('data.html','wb').write(html_doc.content)
    print("File has been created",", ","space = ",file,"bytes")

    read_file = open('data.html','r', encoding="utf-8")
    data = read_file.read()
    
    soup = BeautifulSoup.BeautifulSoup(data,'html')
    table_count = soup.find_all('table')[table_num]
    
    ######################################################
    ## a counter to count the range of columns and rows ##
    rws = 0
    for row in table_count.find_all('tr'):
        cls = 0
        columns = row.find_all('td')
        for column in columns:
            cls += 1
        rws += 1
    ######################################################
    
    table = soup.find_all('table')[table_num]
    
    new_df = pd.DataFrame(columns=range(0, (cls + add_col)), index = range(0, rws))

    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            new_df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        row_marker += 1
    
    print("Dataframe created")
    return new_df


### ~ fetching data from url into a pandas dataframe

In [3]:
# TORONTO
df_t = htmltodf("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", 0)

# NEW YORK CITY
df_nyc = htmltodf("https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm", 0, 1)

# SAN FRANCISCO
df_sf = htmltodf("http://www.healthysf.org/bdi/outcomes/zipmap.htm", 3)

File has been created ,  space =  80694 bytes
Dataframe created
File has been created ,  space =  20586 bytes
Dataframe created
File has been created ,  space =  21606 bytes
Dataframe created


In [4]:
# importing lat and long values
latlongdf_t = pd.DataFrame(pd.read_csv('./Geospatial_Coordinates.csv'))
latlongdf = pd.DataFrame(pd.read_csv("./ZipCode_Lat_Long.csv"))

## ~ cleaning city data

### Toronto Data

In [5]:
df_t[2] = df_t[2].str.rstrip()

# Removing 'Not Assigned' boroughs
df_t = df_t[df_t[1] != "Not assigned"]

# Assigning Boroughs to 'Not Assigned' neighborhoods
df_t[2] = df_t[2].replace('Not assigned', df_t[1])

# Grouping by Neighborhood
df_t = df_t.groupby(df_t[0]).agg(lambda x: ", ".join(list(set(x.tolist()))))

# Resetting the index
df_t = df_t.reset_index()

# Renaming columns
df_t = df_t.rename(columns = {0:"Postal Code", 1:"Borough", 2:"Neighborhood"})

# Filtering for boroughs exclusively in the Toronto area
df_t = df_t[df_t['Borough'].str.contains('Toronto')]

# Merging zip-code and coordinates into final dataframe
df_t = pd.merge(df_t, latlongdf_t, on = "Postal Code")

# Resetting the index
df_t = df_t.reset_index(drop=True)

df_t.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### New York Data

In [6]:
# Inserting row to shift data into the right format
indices = [2, 9, 20, 30, 40]
offset = 0

for i in indices:
    line = pd.DataFrame({0:df_nyc.iloc[((i-1)+offset), 1], 1:df_nyc.iloc[((i-1)+offset), 2], 2:'NaN'}, index=[i])
    df_nyc = pd.concat([df_nyc.iloc[:(i+offset)], line, df_nyc.iloc[(i+offset):]]).reset_index(drop=True)
    offset += 1


# Dropping first row
df_nyc = df_nyc.drop(df_nyc.index[0])


# Adding borough names to the neighborhood
ranges = [(1, 8), (9, 20), (21, 31), (32, 42), (43, 47)]
boroughs_nyc = [1, 9, 21, 32, 43]

for i in range(0, 5):
    df_nyc.iloc[ranges[i][0]:ranges[i][1], 2] = df_nyc.iloc[boroughs_nyc[i]-1, 0]


# Dropping unrequired rows
offset2 = 0
for n in indices:
    df_nyc = df_nyc.drop(df_nyc.index[(n-2)])
    offset2 += 1

    
# Splitting up zip code list into a new dataframe for each neighborhood and
# combining all of the dataframes
df_nyc[1] = df_nyc[1].str.lstrip()
df_length = len(df_nyc)
df_zip = pd.DataFrame()

for row in range(0, df_length):
    zip_list = df_nyc.iat[row, 1].split(",")
    length = len(zip_list)

    df_nyc2 = pd.DataFrame({0:df_nyc.iat[row, 0], 1:zip_list, 2:df_nyc.iat[row, 2]})
    df_zip = pd.concat([df_zip, df_nyc2])


# Resetting index
df_zip = df_zip.reset_index(drop=True)


# Merging NEW YORK CITY zip codes and coordinates into a final dataframe 
df_zip = df_zip.rename(columns = {1:"ZIP"})
df_zip['ZIP'] = df_zip['ZIP'].astype(int)
df_nyc = pd.merge(df_zip, latlongdf, on = 'ZIP')

# Reordering and renaming columns
collist_nyc = ['ZIP', 2, 0, 'LAT', 'LNG']
df_nyc = df_nyc[collist_nyc]
df_nyc = df_nyc.rename(columns = {'ZIP':'Postal Code', 2:'Borough', 0:'Neighborhood', 'LAT':'Latitude', 'LNG':'Longitude'})

df_nyc.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,10453,Bronx,Central Bronx,40.852779,-73.912332
1,10457,Bronx,Central Bronx,40.847150,-73.898680
2,10460,Bronx,Central Bronx,40.841758,-73.879571
3,10458,Bronx,Bronx Park and Fordham,40.862543,-73.888143
4,10467,Bronx,Bronx Park and Fordham,40.869953,-73.865746


In [7]:
# Grouping by neighborhood - avg of latitude and longitude is taken
df_nyc_coordinates = df_nyc.drop(columns = 'Postal Code', axis = 1)
df_nyc_coordinates = df_nyc_coordinates.groupby('Neighborhood').mean()
df_nyc_coordinates = df_nyc_coordinates.reset_index()

df_nyc_coordinates.head(5)

,Neighborhood,Latitude,Longitude
0,Borough Park,40.629269,-73.980661
1,Bronx Park and Fordham,40.866863,-73.884540
2,Bushwick and Williamsburg,40.699151,-73.930459
3,Canarsie and Flatlands,40.630743,-73.897287
4,Central Bronx,40.847229,-73.896861


## San Francisco Data

In [8]:
# Dropping census columns and unecessary rows
df_sf = df_sf.drop(2, axis = 1)
df_sf = df_sf.drop(df_sf.index[0])
df_sf = df_sf.drop(df_sf.index[-1])

# Cleaning neighborhood columns data
sf_length = len(df_sf)
for i in range(0, sf_length):
    df_sf.iloc[i, 1] = ' '.join(df_sf.iloc[i, 1].split())

# Renaming column for the merge function
df_sf = df_sf.rename(columns = {0:"ZIP"})

# Chaning data type of "zip" column 
df_sf['ZIP'] = df_sf['ZIP'].astype(int)

# Merging dataframe with latitude and longitude data
df_sf = pd.merge(df_sf, latlongdf, on = 'ZIP')
df_sf.head(10)

# Renaming columns in final data frame
df_sf = df_sf.rename(columns = {'ZIP':'Postal Code', 1:'Neighborhood', 'LAT':'Latitude', 'LNG':'Longitude'})

df_sf.head(5)

,Postal Code,Neighborhood,Latitude,Longitude
0,94102,Hayes Valley/Tenderloin/North of Market,37.779588,-122.419318
1,94103,South of Market,37.773134,-122.411167
2,94107,Potrero Hill,37.760460,-122.399724
3,94108,Chinatown,37.792007,-122.408575
4,94109,Polk/Russian Hill (Nob Hill),37.795388,-122.422453


## Folium Maps

In [9]:
# IMPORTING FOLIUM PACKAGE

import folium
print("Folium package successfully imported!")

Folium package successfully imported!


### ~ creating maps of the each of the cities based on their location data 

#### TORONTO

In [10]:
# TORONTO
map_t = folium.Map(location=[43.6532, -79.3832], zoom_start=10, height = 300, width = 300)

# add markers to map
for pstlcode, lat, lng, borough, neighborhood in zip(df_t['Postal Code'], df_t['Latitude'], df_t['Longitude'], df_t['Borough'], df_t['Neighborhood']):
    
    label = '{}, {}, {}'.format(neighborhood, borough, pstlcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_t)
    
map_t

#### NEW YORK CITY

In [11]:
# NEW YORK CITY
map_nyc = folium.Map(location=[40.7128, -74.0060], zoom_start=9, height = 300, width = 300)

# add markers to map
for pstlcode, lat, lng, borough, neighborhood in zip(df_nyc['Postal Code'], df_nyc['Latitude'], df_nyc['Longitude'], df_nyc['Borough'], df_nyc['Neighborhood']):
    label = '{}, {}, {}'.format(neighborhood, borough, pstlcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_nyc)

map_nyc

#### SAN FRANCISCO

In [12]:
# SAN FRANCISCO 
map_sf = folium.Map(location=[37.7749, -122.4194], zoom_start=10, height = 300, width = 300)

# add markers to map
for pstlcode, lat, lng, neighborhood in zip(df_sf['Postal Code'], df_sf['Latitude'], df_sf['Longitude'], df_sf['Neighborhood']):
    
    label = '{}, {}'.format(neighborhood, pstlcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sf)

map_sf

## Foursquare API

### ~ bringing in venue data through Foursquare API

In [13]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SITK0Y5ZZVIMT511CTKGYRXAMRQLZ2NQLJHASDU2JHOFCLEL
CLIENT_SECRET:JAZC3A2WWMOWJXCXBVCWCQEVVFFPUN3HFC5WBA1M2QVBRFR1


In [14]:
## Function that retrieves category type from given row
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
## Function that gets the details of venues in the specified radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    # limit of number of venues returned by Foursquare API
    LIMIT_venues = 100
    
    # venues present within radius (meters)
    radius = 500
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT_venues)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# VENUES LIST FOR EACH CITY

### TORONTO ###
toronto_venues = getNearbyVenues(names=df_t['Neighborhood'],
                                   latitudes=df_t['Latitude'],
                                   longitudes=df_t['Longitude']
                                  )
print("Toronto complete")
print("\n")


### NEW YORK CITY ###
nyc_venues = getNearbyVenues(names=df_nyc['Neighborhood'],
                             latitudes=df_nyc['Latitude'],
                             longitudes=df_nyc['Longitude']
                            )
print("New York City complete")
print("\n")

### SAN FRANCISCO ###
sf_venues = getNearbyVenues(names=df_sf['Neighborhood'],
                                   latitudes=df_sf['Latitude'],
                                   longitudes=df_sf['Longitude']
                                  )
print("San Fransisco complete")


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East, Moore Park
Deer Park, Rathnelly, Forest Hill SE, Summerhill West, South Hill
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
King, Richmond, Adelaide
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Victoria Hotel, Commerce Court
Roselawn
Forest Hill West, Forest Hill North
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Grange Park, Kensington Market, Chinatown
Island airport, Harbourfront West, Bathurst Quay, CN Tower, King and Spadina, South Niagara, Railway Lands
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Dufferin, Dovercourt Village
Trinity, Little Portugal
Exhibition Place, Parkdale Village, Brockton
High Park, The 

In [17]:
print("TORONTO = ", toronto_venues.shape)
print("NEW YORK CITY = ", nyc_venues.shape)
print("SAN FRANCISCO", sf_venues.shape)

TORONTO =  (1700, 7)
NEW YORK CITY =  (7577, 7)
SAN FRANCISCO (1224, 7)


In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix="", prefix_sep="")


# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood']
sf_onehot['Neighborhood'] = sf_venues['Neighborhood']


# move neighborhood column to the first column
fixed_columns_t = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns_t]

fixed_columns_nyc = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns_nyc]

fixed_columns_sf = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns_sf]


In [19]:
toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
nyc_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Terminal,Airport Tram,American Restaurant,Amphitheater,...,Waste Facility,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
sf_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()

In [23]:
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,Central Bay Street,0.012195,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.012195,0.000000,0.000000,0.012195,0.000000,0.0
3,Christie,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Church and Wellesley,0.011628,0.011628,0.011628,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.011628,0.011628,0.000000,0.011628,0.0


In [24]:
nyc_grouped.head()

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Terminal,Airport Tram,American Restaurant,...,Waste Facility,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Borough Park,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.007874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.015748,0.000000,0.000000,0.000000
1,Bronx Park and Fordham,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.010870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,Bushwick and Williamsburg,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.013514,...,0.0,0.0,0.0,0.0,0.0,0.0,0.006757,0.000000,0.000000,0.013514
3,Canarsie and Flatlands,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.026316,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.026316,0.000000
4,Central Bronx,0.014085,0.0,0.0,0.0,0.014085,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.014085,0.000000,0.000000


In [25]:
sf_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bayview-Hunters Point,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.166667,...,0.0,0.00,0.0,0.00,0.00,0.000000,0.000000,0.0,0.00,0.000000
1,Castro/Noe Valley,0.0,0.00,0.012658,0.0,0.025316,0.0,0.00,0.0,0.012658,...,0.0,0.00,0.0,0.00,0.00,0.025316,0.012658,0.0,0.00,0.025316
2,Chinatown,0.0,0.00,0.000000,0.0,0.020000,0.0,0.00,0.0,0.000000,...,0.0,0.00,0.0,0.01,0.01,0.000000,0.000000,0.0,0.00,0.010000
3,Haight-Ashbury,0.0,0.02,0.000000,0.0,0.010000,0.0,0.01,0.0,0.000000,...,0.0,0.01,0.0,0.01,0.00,0.010000,0.000000,0.0,0.01,0.010000
4,Hayes Valley/Tenderloin/North of Market,0.0,0.00,0.000000,0.0,0.020000,0.0,0.00,0.0,0.000000,...,0.0,0.02,0.0,0.03,0.00,0.020000,0.010000,0.0,0.00,0.000000


In [26]:
## Function that returns the most common venue
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe having columns labelled with appropriate ranks 
neighborhoods_venues_sorted_toronto = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_nyc = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_sf = pd.DataFrame(columns=columns)

# assign grouped dataframes to the newly created dataframes
neighborhoods_venues_sorted_toronto['Neighborhood'] = toronto_grouped['Neighborhood']
neighborhoods_venues_sorted_nyc['Neighborhood'] = nyc_grouped['Neighborhood']
neighborhoods_venues_sorted_sf['Neighborhood'] = sf_grouped['Neighborhood']

# storing the top 5 venues found for each neighborhood in that city to a dataframe
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted_toronto.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
for ind in np.arange(nyc_grouped.shape[0]):
    neighborhoods_venues_sorted_nyc.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)
for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted_sf.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)



In [28]:
neighborhoods_venues_sorted_toronto.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pub,Cheese Shop,Farmers Market,Bakery,Café,Beer Bar,Seafood Restaurant
1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Farmers Market,Spa,Fast Food Restaurant,Brewery,Restaurant,Burrito Place,Recording Studio
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Middle Eastern Restaurant,Thai Restaurant,Salad Place,Bubble Tea Shop,Spa
3,Christie,Café,Grocery Store,Park,Italian Restaurant,Restaurant,Convenience Store,Baby Store,Diner,Nightclub,Coffee Shop
4,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Gastropub,Fast Food Restaurant,Bubble Tea Shop,Men's Store


In [29]:
neighborhoods_venues_sorted_nyc.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Borough Park,Pizza Place,Bakery,Grocery Store,Pharmacy,Bagel Shop,Bank,Chinese Restaurant,Café,Italian Restaurant,Convenience Store
1,Bronx Park and Fordham,Pizza Place,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Sandwich Place,Donut Shop,Fast Food Restaurant,Deli / Bodega,Mexican Restaurant,Café
2,Bushwick and Williamsburg,Bar,Mexican Restaurant,Pizza Place,Coffee Shop,Deli / Bodega,Bakery,Café,Cocktail Bar,Chinese Restaurant,Spanish Restaurant
3,Canarsie and Flatlands,Caribbean Restaurant,Convenience Store,Pizza Place,Gym,Bank,Bus Station,Music Venue,Bagel Shop,Grocery Store,Mobile Phone Shop
4,Central Bronx,Pizza Place,Fast Food Restaurant,Bus Station,Donut Shop,Spanish Restaurant,Supermarket,Sandwich Place,Bank,Deli / Bodega,Liquor Store


In [30]:
neighborhoods_venues_sorted_sf.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview-Hunters Point,Brewery,Lighting Store,Seafood Restaurant,Motorcycle Shop,Bus Station,Art Gallery,Food Truck,Flower Shop,Food,Food & Drink Shop
1,Castro/Noe Valley,Gay Bar,Coffee Shop,Café,Playground,Pet Store,New American Restaurant,Thai Restaurant,Deli / Bodega,Clothing Store,Yoga Studio
2,Chinatown,Hotel,Coffee Shop,French Restaurant,Café,Cocktail Bar,Japanese Restaurant,Church,Pizza Place,Italian Restaurant,Hotel Bar
3,Haight-Ashbury,Boutique,Thrift / Vintage Store,Pizza Place,Café,Coffee Shop,Breakfast Spot,Shoe Store,Bookstore,Board Shop,Gift Shop
4,Hayes Valley/Tenderloin/North of Market,Coffee Shop,Café,Cocktail Bar,French Restaurant,Theater,Beer Bar,Boutique,Furniture / Home Store,Sandwich Place,Vietnamese Restaurant


## Comparison

### ~ initializing KMeans and DBSCAN

In [31]:
# setting up data for algorithms
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
nyc_grouped_clustering = nyc_grouped.drop('Neighborhood', 1)
sf_grouped_clustering = sf_grouped.drop('Neighborhood', 1)

## KMEANS
kclusters = 2
init = 'k-means++'
n_init = 10
max_iter = 300
kmeans_t = KMeans(init=init, n_init=n_init, max_iter=max_iter, n_clusters=kclusters, random_state=(random.seed(42))).fit(toronto_grouped_clustering)
kmeans_nyc = KMeans(init=init, n_init=n_init, max_iter=max_iter, n_clusters=kclusters, random_state=(random.seed(42))).fit(nyc_grouped_clustering)
kmeans_sf = KMeans(init=init, n_init=n_init, max_iter=max_iter, n_clusters=kclusters, random_state=(random.seed(42))).fit(sf_grouped_clustering)

## DBSCAN
eps = 0.5
min_samples = 10
metric = 'euclidean'
algorithm = 'auto'
leaf_size = 30
dbscan_t = DBSCAN(eps=eps, min_samples=min_samples, metric=metric, algorithm=algorithm, leaf_size=leaf_size).fit(toronto_grouped_clustering)
dbscan_nyc = DBSCAN(eps=eps, min_samples=min_samples, metric=metric, algorithm=algorithm, leaf_size=leaf_size).fit(nyc_grouped_clustering)
dbscan_sf = DBSCAN(eps=eps, min_samples=min_samples, metric=metric, algorithm=algorithm, leaf_size=leaf_size).fit(sf_grouped_clustering)


In [32]:
## initializing final dataframe for clustering algorithm data
toronto_merged = df_t
nyc_merged = df_nyc_coordinates
sf_merged = df_sf

## setting cluster labels to column for kmeans and dbscan
toronto_merged['Cluster Labels kmeans'] = kmeans_t.labels_
nyc_merged['Cluster Labels kmeans'] = kmeans_nyc.labels_
sf_merged['Cluster Labels kmeans'] = kmeans_sf.labels_

toronto_merged['Cluster Labels dbscan'] = dbscan_t.labels_
nyc_merged['Cluster Labels dbscan'] = dbscan_nyc.labels_
sf_merged['Cluster Labels dbscan'] = dbscan_sf.labels_

## combining dataframes
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted_toronto.set_index('Neighborhood'), on='Neighborhood')
nyc_merged = nyc_merged.join(neighborhoods_venues_sorted_nyc.set_index('Neighborhood'), on='Neighborhood')
sf_merged = sf_merged.join(neighborhoods_venues_sorted_sf.set_index('Neighborhood'), on='Neighborhood')


### ~ number of clusters and types of venues

In [33]:
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels kmeans,Cluster Labels dbscan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,0,Coffee Shop,Pub,Astrologer,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Brewery,Bakery,Spa,Juice Bar
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,0,Park,Pub,Ice Cream Shop,Board Shop,Sandwich Place,Brewery,Fast Food Restaurant,Fish & Chips Shop,Burger Joint,Burrito Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Bar,Fish Market,Juice Bar,Coworking Space,Latin American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,0,Gym / Fitness Center,Park,Swim School,Bus Line,Concert Hall,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,0,Gym,Clothing Store,Burger Joint,Sandwich Place,Food & Drink Shop,Hotel,Breakfast Spot,Park,Grocery Store,Dumpling Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,0,Coffee Shop,Sporting Goods Shop,Yoga Studio,Spa,Dessert Shop,Mexican Restaurant,Salon / Barbershop,Sandwich Place,Fast Food Restaurant,Chinese Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,0,Pizza Place,Sandwich Place,Dessert Shop,Café,Sushi Restaurant,Italian Restaurant,Seafood Restaurant,Coffee Shop,Toy / Game Store,Greek Restaurant
8,M4T,Central Toronto,"Summerhill East, Moore Park",43.689574,-79.383160,1,0,Restaurant,Playground,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,M4V,Central Toronto,"Deer Park, Rathnelly, Forest Hill SE, Summerhi...",43.686412,-79.400049,1,0,Pub,Coffee Shop,American Restaurant,Bagel Shop,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station


In [34]:
nyc_merged

,Neighborhood,Latitude,Longitude,Cluster Labels kmeans,Cluster Labels dbscan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Borough Park,40.629269,-73.980661,1,0,Pizza Place,Bakery,Grocery Store,Pharmacy,Bagel Shop,Bank,Chinese Restaurant,Café,Italian Restaurant,Convenience Store
1,Bronx Park and Fordham,40.866863,-73.884540,1,0,Pizza Place,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Sandwich Place,Donut Shop,Fast Food Restaurant,Deli / Bodega,Mexican Restaurant,Café
2,Bushwick and Williamsburg,40.699151,-73.930459,1,0,Bar,Mexican Restaurant,Pizza Place,Coffee Shop,Deli / Bodega,Bakery,Café,Cocktail Bar,Chinese Restaurant,Spanish Restaurant
3,Canarsie and Flatlands,40.630743,-73.897287,1,0,Caribbean Restaurant,Convenience Store,Pizza Place,Gym,Bank,Bus Station,Music Venue,Bagel Shop,Grocery Store,Mobile Phone Shop
4,Central Bronx,40.847229,-73.896861,1,0,Pizza Place,Fast Food Restaurant,Bus Station,Donut Shop,Spanish Restaurant,Supermarket,Sandwich Place,Bank,Deli / Bodega,Liquor Store
5,Central Brooklyn,40.674452,-73.936484,1,0,Coffee Shop,Bar,Caribbean Restaurant,Café,Mexican Restaurant,Deli / Bodega,Grocery Store,Cocktail Bar,Pizza Place,Sandwich Place
6,Central Harlem,40.815176,-73.944438,1,0,Southern / Soul Food Restaurant,Seafood Restaurant,Coffee Shop,Deli / Bodega,American Restaurant,Pizza Place,Park,Wine Shop,Cosmetics Shop,Café
7,Central Queens,40.732644,-73.802180,1,0,Chinese Restaurant,Pizza Place,Bus Station,Bagel Shop,Convenience Store,Donut Shop,Sushi Restaurant,Coffee Shop,Food Truck,Middle Eastern Restaurant
8,Chelsea and Clinton,40.755218,-73.991129,1,0,Theater,Coffee Shop,Italian Restaurant,Hotel,Burger Joint,American Restaurant,Pizza Place,Dance Studio,Steakhouse,Sandwich Place
9,East Harlem,40.793609,-73.936813,1,0,Mexican Restaurant,Bakery,Pizza Place,Deli / Bodega,Clothing Store,Sandwich Place,Latin American Restaurant,Supermarket,Chinese Restaurant,Trail


In [35]:
sf_merged

,Postal Code,Neighborhood,Latitude,Longitude,Cluster Labels kmeans,Cluster Labels dbscan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94102,Hayes Valley/Tenderloin/North of Market,37.779588,-122.419318,0,0,Coffee Shop,Café,Cocktail Bar,French Restaurant,Theater,Beer Bar,Boutique,Furniture / Home Store,Sandwich Place,Vietnamese Restaurant
1,94103,South of Market,37.773134,-122.411167,0,0,Nightclub,Cocktail Bar,Gay Bar,Food Truck,Coffee Shop,Café,Thai Restaurant,Motorcycle Shop,Art Gallery,Furniture / Home Store
2,94107,Potrero Hill,37.760460,-122.399724,0,0,Park,Café,Playground,Breakfast Spot,Brewery,Garden,Yoga Studio,Sushi Restaurant,Mac & Cheese Joint,Bookstore
3,94108,Chinatown,37.792007,-122.408575,0,0,Hotel,Coffee Shop,French Restaurant,Café,Cocktail Bar,Japanese Restaurant,Church,Pizza Place,Italian Restaurant,Hotel Bar
4,94109,Polk/Russian Hill (Nob Hill),37.795388,-122.422453,0,0,Gym / Fitness Center,Italian Restaurant,Deli / Bodega,Bar,Cosmetics Shop,Wine Bar,Coffee Shop,Spa,Gym,Sushi Restaurant
5,94110,Inner Mission/Bernal Heights,37.750021,-122.415201,0,0,Mexican Restaurant,Coffee Shop,Grocery Store,Pizza Place,Art Gallery,Bakery,Gym / Fitness Center,Café,Latin American Restaurant,Bookstore
6,94112,Ingelside-Excelsior/Crocker-Amazon,37.720375,-122.442950,0,0,Pizza Place,Sandwich Place,Mexican Restaurant,Vietnamese Restaurant,Bus Station,Filipino Restaurant,Restaurant,Chinese Restaurant,Flower Shop,Thai Restaurant
7,94114,Castro/Noe Valley,37.758057,-122.435410,0,0,Gay Bar,Coffee Shop,Café,Playground,Pet Store,New American Restaurant,Thai Restaurant,Deli / Bodega,Clothing Store,Yoga Studio
8,94115,Western Addition/Japantown,37.785969,-122.437253,0,0,Cosmetics Shop,Bakery,Spa,Ice Cream Shop,Yoga Studio,Café,Boutique,Tea Room,Italian Restaurant,Gym / Fitness Center
9,94116,Parkside/Forest Hill,37.745399,-122.486065,0,0,Chinese Restaurant,Dumpling Restaurant,Japanese Restaurant,Liquor Store,Café,Spa,Burrito Place,Shoe Store,Shoe Repair,Sandwich Place


In [36]:
## finding out the number of clusters that dbscan created
clusters_db = []
clusters_db.append(toronto_merged['Cluster Labels dbscan'].unique())
clusters_db.append(nyc_merged['Cluster Labels dbscan'].unique())
clusters_db.append(sf_merged['Cluster Labels dbscan'].unique())

print("ndbscan = {}".format(clusters_db))

ndbscan = [array([ 0, -1], dtype=int64), array([0], dtype=int64), array([ 0, -1], dtype=int64)]


In [37]:
# setting columns names for most common venues
col_names = []
col_names = toronto_merged.columns
col_names = col_names[7:]

# initializing dictionaries for each city 
dr_toronto = {}
dr_nyc = {}
dr_sf = {}

# running for loop to store count, unique, top, and freq values of the 'describe' function
for i in col_names:
    
    if i not in dr_toronto:
        dr_toronto[i] = []
        dr_toronto[i].append(toronto_merged[i].describe()[0])
        dr_toronto[i].append(toronto_merged[i].describe()[1])
        dr_toronto[i].append(toronto_merged[i].describe()[2])
        dr_toronto[i].append(toronto_merged[i].describe()[3])
        
    if i not in dr_nyc:
        dr_nyc[i] = []
        dr_nyc[i].append(nyc_merged[i].describe()[0])
        dr_nyc[i].append(nyc_merged[i].describe()[1])
        dr_nyc[i].append(nyc_merged[i].describe()[2])
        dr_nyc[i].append(nyc_merged[i].describe()[3])
        
    if i not in dr_sf:
        dr_sf[i] = []
        dr_sf[i].append(sf_merged[i].describe()[0])
        dr_sf[i].append(sf_merged[i].describe()[1])
        dr_sf[i].append(sf_merged[i].describe()[2])
        dr_sf[i].append(sf_merged[i].describe()[3])

# storing data to dataframes
compare_t = pd.DataFrame(dr_toronto)
compare_nyc = pd.DataFrame(dr_nyc)
compare_sf = pd.DataFrame(dr_sf)

# setting up final comparison dataframe
compare = pd.concat([compare_t, compare_nyc, compare_sf])
compare = compare.reset_index(drop=True)
compare = compare.transpose()
compare = compare.rename(columns = {0:"Toronto", 1:"Unique_t", 2:"TopVenue_t", 3:"Freq_t",
                                   4:"NYC", 5:"Unique_nyc", 6:"TopVenue_nyc", 7:"Freq_nyc",
                                   8:"SF", 9:"Unique_sf", 10:"TopVenue_sf", 11:"Freq_sf"})
compare

,Toronto,Unique_t,TopVenue_t,Freq_t,NYC,Unique_nyc,TopVenue_nyc,Freq_nyc,SF,Unique_sf,TopVenue_sf,Freq_sf
1st Most Common Venue,38,17,Coffee Shop,16,42,15,Pizza Place,11,21,19,Coffee Shop,2
2nd Most Common Venue,38,20,Café,6,42,21,Pizza Place,7,21,16,Café,3
3rd Most Common Venue,38,24,Hotel,5,42,22,Chinese Restaurant,6,21,20,Café,2
4th Most Common Venue,38,28,Restaurant,4,42,23,Pharmacy,4,21,18,Playground,2
5th Most Common Venue,38,28,American Restaurant,5,42,26,Deli / Bodega,4,21,19,Coffee Shop,2
6th Most Common Venue,38,29,Restaurant,3,42,26,Donut Shop,6,21,20,Café,2
7th Most Common Venue,38,31,Brewery,3,42,29,Pizza Place,4,21,17,Boutique,2
8th Most Common Venue,38,30,Bakery,3,42,29,Bakery,3,21,20,Flower Shop,2
9th Most Common Venue,38,33,Bubble Tea Shop,2,42,32,Pizza Place,3,21,18,Food,3
10th Most Common Venue,38,33,Eastern European Restaurant,3,42,32,Mexican Restaurant,4,21,19,Food & Drink Shop,2


## END